In [1]:
import re
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import datediff, col, lag, avg
from pyspark.sql.window import Window

In [2]:
sp = SparkSession.builder.appName("postcode analysis").config("spark.sql.session.timeZone", "+10").getOrCreate()
sp

22/10/02 17:32:27 WARN Utils: Your hostname, J-L resolves to a loopback address: 127.0.1.1; using 172.17.236.253 instead (on interface eth0)
22/10/02 17:32:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/02 17:32:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read all transactions together
from functools import reduce

dir = "../data/tables/"
groups = ["transactions_20210228_20210827_snapshot/", "transactions_20210828_20220227_snapshot/", "transactions_20220228_20220828_snapshot/"]
final_list = []

for g in groups:
    final_list.append(sp.read.parquet(dir + g))

trans = reduce(DataFrame.unionAll, final_list)
trans.show(4)

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  18478| 62191208634|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|
|      2| 15549624934| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|
|  18479| 64403598239|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|
|      3| 60956456424| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|
+-------+------------+------------------+--------------------+--------------+
only showing top 4 rows



In [7]:
trans.filter(trans.user_id == 880).groupBy("merchant_abn").count().show()

+------------+-----+
|merchant_abn|count|
+------------+-----+
| 49891706470|    9|
| 91772719127|    1|
| 89726005175|   10|
| 11173270682|    1|
| 96152467973|    4|
| 74391255526|    1|
| 39112692726|    1|
| 94072762560|    2|
| 86578477987|    7|
| 45244796758|    1|
| 32709545238|    1|
| 29521780474|    1|
| 32361057556|    3|
| 92990617267|    1|
| 62204398944|    2|
| 51808433866|    1|
| 75944642726|    1|
| 56144254367|    1|
| 53965592614|    1|
| 90918180829|    1|
+------------+-----+
only showing top 20 rows



In [9]:
consumer_data = sp.read.option("header", True).option("delimiter", "|").csv("../data/tables/tbl_consumer.csv")
consumer_data.show(2)

+----------------+--------------------+-----+--------+------+-----------+
|            name|             address|state|postcode|gender|consumer_id|
+----------------+--------------------+-----+--------+------+-----------+
|Yolanda Williams|413 Haney Gardens...|   WA|    6935|Female|    1195503|
|      Mary Smith|     3764 Amber Oval|  NSW|    2782|Female|     179208|
+----------------+--------------------+-----+--------+------+-----------+
only showing top 2 rows



In [12]:
consumer_data.filter(consumer_data.postcode == 1819).show()

+--------------------+--------------------+-----+--------+-----------+-----------+
|                name|             address|state|postcode|     gender|consumer_id|
+--------------------+--------------------+-----+--------+-----------+-----------+
|         Molly Perez| 489 Benitez Heights|  NSW|    1819|     Female|     114419|
|      Jessica Hansen|     8537 Jason Fork|  NSW|    1819|     Female|     824107|
|       Larry Gilbert|7060 Gilbert Port...|  NSW|    1819|       Male|      15019|
|      Jamie Williams|17563 Becker Shoa...|  NSW|    1819|     Female|     805309|
|    Richard Richards|020 Jasmine Branc...|  NSW|    1819|Undisclosed|    1452640|
|      Candice Wright|21199 Ford Flat S...|  NSW|    1819|     Female|     774094|
|   Mackenzie Jackson|   77624 James Drive|  NSW|    1819|     Female|     716474|
|          John Walsh|50604 Brown Brook...|  NSW|    1819|       Male|     629028|
|Christopher Cunni...|   2870 Moyer Spring|  NSW|    1819|       Male|    1171474|
|   